# SQL. Старт. Зима-весна 2024

Для решения сгенерированы данные при помощи Python (Приложение 1).  
Проверка решения проводилась https://sqliteonline.com/ в СУБД PostgreSQL (Приложение 2).  


1.	Напишите запрос, с помощью которого можно найти дубли в поле email из таблицы “Sfaff”.

SELECT email AS double_email FROM "Staff"  
GROUP BY email  
HAVING COUNT(email)>  1;


2. Напишите запрос, с помощью которого можно определить возраст каждого сотрудника из таблицы “Staff” на момент запроса.

SELECT name,  EXTRACT(YEAR FROM AGE(CURRENT_DATE, TO_DATE(bithday, 'DD.MM.YYYY'))) AS years_old  
FROM "Staff"  
WHERE bithday IS NOT NULL

3. Напишите запрос, с помощью которого можно определить должность (Jobtitles.name) со вторым по величине уровнем зарплаты.

Решение 1.  
SELECT name FROM "Jobtitles"  
WHERE jobtitle_id = (SELECT jobtitle_id FROM "Staff" WHERE salary is NOT NULL ORDER BY salary DESC OFFSET 1 LIMIT 1) 

Решение 2. Учитывает одинаковый уровень зарплаты у нескольких сотрудников.  
WITH cte AS  
( SELECT salary, jobtitle_id, dense_rank() OVER(ORDER BY salary DESC) AS salary_level  
FROM "Staff"  
WHERE salary IS NOT NULL) 

SELECT name  
FROM "Jobtitles"  
WHERE jobtitle_id in (SELECT jobtitle_id FROM cte WHERE salary_level=2)

### Приложение 1. 
Python get data scrip


In [ ]:
import pandas as pd
from random import choice, randint
from string import ascii_lowercase
from datetime import datetime as dt

data = pd.DataFrame()
n = 100

data["staff_id"] = list(range(1, n+1))

name = ["James", "John", "Robert", "Michae", 
        "William", "Charles", "Richard", "Paul", 
        "Christopher", "David", "Donald", ]
surname = ["Ross", "Ramirez", "Sanders", "Bailey", 
           "Morris", "Gonzalez", "Hill", "Hernandez", "Walker" ]

data["name"] = [f"{choice(name)} {choice(surname)}" for _ in range(n)]

data["salary"] = [randint(27_894, 1_047_587) for _ in range(n)]

def get_email():
    l = randint(7, 13)
    txt = "".join(choice(ascii_lowercase) for _ in range(l))
    return f"{txt}@{choice(domain)}"
domain = ["mail.ru", "ya.ru", "yandex.ru", "gmail.com", "rambler.ru", "bk.ru", "list.ru"]
data["email"] = [get_email() for _ in range(n)]

def get_bithday():
    start_date = dt(randint(1985, 2005), randint(1,12), randint(1, 28))
    return start_date.strftime("%d.%m.%Y")
data["bithday"] = [get_bithday() for _ in range(n)]

data["jobtitle_id"] = [randint(1, 7) for _ in range(n)]

data_2 = pd.DataFrame({"jobtitle_id": list(range(1,8)),
                      "name": ["Developer", "System Analyst", "Project Manager", 
                               "System Administrator", "Group Leader", 
                               "Testing Engineer", "Support", ]
                      })
data_2

data.to_csv("Staff.csv", index=False)
data_2.to_csv("Jobtitles.csv", index=False)


### Приложение 2. 
Подготовка таблиц для запросов SQL


CREATE TABLE "Jobtitles" ("jobtitle_id" SMALLINT,"name" VARCHAR(20));  
INSERT INTO "Jobtitles" ("jobtitle_id","name") VALUES  
('1','Developer'),  
('2','System Analyst'),  
('3','Project Manager'),  
('4','System Administrator'),  
('5','Group Leader'),  
('6','Testing Engineer'),  
('7','Support'); 

CREATE TABLE "Staff" ("staff_id" SMALLINT,"name" VARCHAR(30),"salary" INTEGER,"email" VARCHAR(30),"bithday" VARCHAR(10),"jobtitle_id" SMALLINT);  
INSERT INTO "Staff" ("staff_id","name","salary","email","bithday","jobtitle_id") VALUES  
('1','William Hernandez','616342','abfnnta@mail.ru','06.01.1986','6'),  
('2','David Morris','747609','fwfswem@rambler.ru','08.03.1986','2'),  
('3','Richard Ramirez','656008','wemxyzljgjhsv@bk.ru','20.03.2002','4'),  ……………………………………………………………………………………………………………………………………………………….  
('98','Paul Ross','977718','pqrmwyurjaap@list.ru','25.04.2003','4'),  
('99','Christopher Hill','268727','wnuxswmldwbf@bk.ru','27.03.1992','7'),  
('100','Michae Morris','245944','kseewamtqtm@list.ru','14.06.1997','2'); 

В генерации отсутствуют NULL, поэтому:  
UPDATE "Staff"  
SET email = NULL  
WHERE staff_id % 13 = 0;  
SELECT * FROM "Staff"; 

UPDATE "Staff"  
SET bithday = NULL  
WHERE (staff_id-5) % 29 = 0;  

UPDATE "Staff"  
SET salary = NULL  
WHERE (staff_id+13) % 29 = 0; 

В генерации отсутствуют дубли, поэтому:  
UPDATE "Staff"  
SET email = (SELECT email FROM "Staff" WHERE staff_id=15 )  
WHERE staff_id IN (20,40); 

UPDATE "Staff"  
SET email = (SELECT email FROM "Staff" WHERE staff_id=3 )  
WHERE staff_id = 50;